In [74]:
pip install flask flask_sqlalchemy pandas openpyxl flask_mail


Note: you may need to restart the kernel to use updated packages.


In [75]:
from flask import Flask, render_template, request, redirect, url_for, flash
from flask_sqlalchemy import SQLAlchemy
import pandas as pd
from datetime import datetime, timedelta
from flask_mail import Mail, Message
from sqlalchemy import or_

# Initialisation de l'application Flask
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.secret_key = "supersecretkey"

# Configuration de Flask-Mail
app.config['MAIL_SERVER'] = 'smtp.gmail.com'
app.config['MAIL_PORT'] = 587
app.config['MAIL_USE_TLS'] = True
app.config['MAIL_USERNAME'] = 'tonemail@gmail.com'  # Remplace par ton email
app.config['MAIL_PASSWORD'] = 'tonmotdepasse'  # Remplace par ton mot de passe
mail = Mail(app)

db = SQLAlchemy(app)

# ----------------------------
# Définition des modèles
# ----------------------------

class Benevole(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    nom = db.Column(db.String(100), nullable=False)
    prenom = db.Column(db.String(100), nullable=False)
    email = db.Column(db.String(100), unique=True, nullable=False)
    telephone = db.Column(db.String(20), nullable=False)
    activite_count = db.Column(db.Integer, default=0)
    dernieres_activites = db.Column(db.DateTime, nullable=True)

class Mission(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    titre = db.Column(db.String(200), nullable=False)
    description = db.Column(db.Text, nullable=False)
    nb_places = db.Column(db.Integer, nullable=False)

class Inscription(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    benevole_id = db.Column(db.Integer, db.ForeignKey('benevole.id'), nullable=False)
    mission_id = db.Column(db.Integer, db.ForeignKey('mission.id'), nullable=False)
    date_inscription = db.Column(db.DateTime, default=datetime.utcnow)

with app.app_context():
    db.create_all()


In [76]:
@app.route('/import', methods=['GET', 'POST'])
def import_excel():
    if request.method == 'POST':
        file = request.files['file']
        if not file:
            flash('❌ Aucun fichier sélectionné.', 'danger')
            return redirect(request.url)

        # Charger le fichier Excel
        file_path = file
        df = pd.read_excel(file_path)

        # S'assurer que les bonnes colonnes sont présentes
        if not {'nom', 'prenom', 'email', 'telephone'}.issubset(df.columns):
            flash('❌ Format de fichier incorrect.', 'danger')
            return redirect(request.url)

        # Utilisation des colonnes pertinentes
        df_benevoles = df[['Prénom', 'Nom ', 'Numéro de Téléphone', 'Ton adresse mail ?']] 
        df_benevoles.columns = ['prenom', 'nom', 'telephone', 'email']

        # Importation dans la base de données
        for _, row in df_benevoles.iterrows():
            # Vérification des valeurs essentielles (nom, prénom, email ou téléphone requis)
            if pd.notna(row['nom']) and pd.notna(row['prenom']):
                nom = str(row['nom']).strip()
                prenom = str(row['prenom']).strip()
                email = str(row['email']).strip() if pd.notna(row['email']) else None
                telephone = None

                if pd.notna(row['telephone']):  
                    try:
                        telephone = str(int(row['telephone']))  # Convertir en entier puis en str
                    except ValueError:
                        telephone = None  # Si conversion impossible, on met None

                # Si aucun téléphone, on passe à la ligne suivante
                if not telephone:
                    continue  

                # Vérifier si l'email ou le téléphone existent déjà
                conditions = []
                if email:
                    conditions.append(Benevole.email == email)
                if telephone:
                    conditions.append(Benevole.telephone == telephone)

                exists = Benevole.query.filter(or_(*conditions)).first() if conditions else None

                if not exists:
                    benevole = Benevole(
                        prenom=prenom,
                        nom=nom,
                        email=email,
                        telephone=telephone
                    )
                    db.session.add(benevole)  # Ajout du bénévole à la session

        db.session.commit()  # Validation des ajouts dans la base de données
        flash('✅ Données importées avec succès !', 'success')
        return redirect(url_for('missions'))

    return render_template('import.html')


In [77]:
@app.route('/missions', methods=['GET', 'POST'])
def missions():
    if request.method == 'POST':
        titre = request.form['titre']
        description = request.form['description']
        nb_places = request.form['nb_places']

        mission = Mission(titre=titre, description=description, nb_places=int(nb_places))
        db.session.add(mission)
        db.session.commit()
        flash('✅ Mission créée avec succès !', 'success')

    missions = Mission.query.all()
    return render_template('missions.html', missions=missions)


In [78]:
@app.route('/inscription/<int:mission_id>/<int:benevole_id>')
def inscrire_benevole(mission_id, benevole_id):
    inscription = Inscription(mission_id=mission_id, benevole_id=benevole_id)
    benevole = Benevole.query.get(benevole_id)
    benevole.activite_count += 1
    benevole.dernieres_activites = datetime.utcnow()

    db.session.add(inscription)
    db.session.commit()
    flash('✅ Inscription réussie !', 'success')

    return redirect(url_for('missions'))


In [79]:
@app.route('/classement')
def classement():
    benevoles = Benevole.query.order_by(Benevole.activite_count.desc()).all()
    return render_template('classement.html', benevoles=benevoles)


In [80]:
#delete the non active volunteer
@app.route('/cleanup')
def cleanup():
    three_months_ago = datetime.utcnow() - timedelta(days=90)
    inactive_benevoles = Benevole.query.filter(Benevole.dernieres_activites < three_months_ago).all()

    for benevole in inactive_benevoles:
        db.session.delete(benevole)

    db.session.commit()
    return "Bénévoles inactifs supprimés !"


In [88]:
#arning if there is no activity after 2 months 
@app.route('/send_warnings')
def send_warnings():
    two_months_ago = datetime.utcnow() - timedelta(days=60)
    benevoles = Benevole.query.filter(Benevole.dernieres_activites < two_months_ago).all()

    for benevole in benevoles:
        msg = Message("Avertissement d'inactivité", sender="tonemail@gmail.com", recipients=[benevole.email])
        msg.body = f"Bonjour {benevole.nom},\n\nVous êtes inactif depuis 2 mois. Si vous ne participez pas à une mission dans le mois qui suit, vous serez retiré de la liste des bénévoles."
        mail.send(msg)

    return "Emails envoyés !"
